In [1]:
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)
os.chdir('..')

In [2]:
import pandas as pd
from tqdm.notebook import tqdm

/tmp/ipykernel_28052/1669875576.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# loading data

In [3]:
day = '2023-12-20'

columns = ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume']
types = ['str'] * 2 + ['float64'] * 4 + ['int']
dtype = dict(zip(range(len(types)), types))
dtype

{0: 'str',
 1: 'str',
 2: 'float64',
 3: 'float64',
 4: 'float64',
 5: 'float64',
 6: 'int'}

In [4]:
all_stock_files = sorted(os.listdir('data/ticker_data'))
all_stocks = list(map(lambda x: x.replace('.csv', ''), all_stock_files))
len(all_stocks)

446

In [5]:
# all_stock_files = os.listdir('data/ticker_data')
# for file in tqdm(all_stock_files):
#     try:
#         dataset = pd.read_csv(f'data/ticker_data/{file}')
#     except:
#         os.remove(f'data/ticker_data/{file}')

In [6]:
# from src.finam_download import get

# try:
#     get('ACGL', 'M1', f'data/ticker_data/ACGL.csv')
# except:
#     pass

In [7]:
df = pd.DataFrame()

for ticker in tqdm(all_stocks[:10]):
    try:
        dataset = pd.read_csv(f'data/ticker_data/{ticker}.csv', header=None,  dtype=dtype)
        dataset.columns = columns

        dataset['Datetime'] = dataset['Date'].astype('str') + '-' + dataset['Time'].astype('str')
        dataset['Datetime'] = pd.to_datetime(dataset['Datetime'], format='%Y%m%d-%H%M%S')
        dataset['Stock'] = ticker

        df = pd.concat([df, dataset])
    except:
        print(ticker)

df.head()

  0%|          | 0/10 [00:00<?, ?it/s]

,Date,Time,Open,High,Low,Close,Volume,Datetime,Stock
0,20240130,235900,134.35,134.36,134.30,134.30,992,2024-01-30 23:59:00,A
1,20240130,235700,134.34,134.36,134.31,134.31,830,2024-01-30 23:57:00,A
2,20240130,235600,134.28,134.30,134.28,134.30,300,2024-01-30 23:56:00,A
3,20240130,235500,134.33,134.33,134.26,134.27,1207,2024-01-30 23:55:00,A
4,20240130,235400,134.35,134.35,134.35,134.35,100,2024-01-30 23:54:00,A


In [10]:
df.pivot(index='Datetime', columns='Stock', values='Close')

Stock,A,AAL,AAPL,ABBV,ABNB,ABT,ACN,ADBE,ADI,ADM
Datetime,,,,,,,,,,
2023-01-30 17:30:00,NaN,16.270,145.13,145.87,114.690,109.760,NaN,367.000,NaN,83.190
2023-01-30 17:31:00,NaN,16.265,145.32,145.52,114.520,109.850,NaN,NaN,NaN,82.880
2023-01-30 17:32:00,154.84,16.190,145.37,145.53,114.635,110.050,275.15,365.975,168.980,82.790
2023-01-30 17:33:00,NaN,16.260,144.88,145.55,114.030,109.980,NaN,366.280,169.005,82.740
2023-01-30 17:34:00,154.92,16.250,144.61,NaN,113.720,109.930,NaN,NaN,169.185,82.630
...,...,...,...,...,...,...,...,...,...,...
2024-01-30 23:55:00,134.27,14.545,187.95,164.74,149.640,113.360,371.29,627.880,194.770,56.050
2024-01-30 23:56:00,134.30,14.545,188.02,164.75,149.680,113.360,371.15,NaN,194.800,56.030
2024-01-30 23:57:00,134.31,14.535,187.92,164.75,149.550,113.380,371.12,NaN,194.870,55.985
